In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [2]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from numpy.random import seed
from sklearn.utils import class_weight

from inception_model_generator_BN import MRNet_inc_model

import tensorflow as tf


Using TensorFlow backend.


#trainning on augmented balanced data with only the skewed class augmented
20 epochs


In [0]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(1,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_weight = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l , batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)

model.fit(x=train_data_gen, epochs=100, initial_epoch = 50,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

In [0]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(1,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)

model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN_smalllr/meniscus/axial/weights.50.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=50, initial_epoch = 0,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

Tensor("sequential_1/mr_net_inception_layer_1/dropout_1/cond/Identity:0", shape=(1, 2048), dtype=float32)
Tensor("sequential_1/mr_net_inception_layer_1_1/dropout_1/cond/Identity:0", shape=(1, 2048), dtype=float32)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_1 (MR (None, 1)                 21691809  
Total params: 21,691,809
Trainable params: 21,622,945
Non-trainable params: 68,864
_________________________________________________________________
Initializing Data Generator:
factor :  4.242268041237113
model:  inception
data type:  train
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1646
input size:  (299, 299)
Initializing Data Generator:
factor :  3.9130434782608696
model:  inception
data type:  test
Combination:  abnormal  and  axial
data path:  /content/gdrive/M

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 0 could not be retrieved. It could be because a worker has died.
  UserWarning)


1646/1646 [==============================] - 788s 479ms/step - loss: 0.5813 - tp: 3962.0220 - fp: 3438.5923 - tn: 618.2266 - fn: 108.6592 - accuracy: 0.5631 - precision: 0.5351 - recall: 0.9733 - auc: 0.7052 - val_loss: 0.6936 - val_tp: 4401.7002 - val_fp: 3715.8000 - val_tn: 801.0000 - val_fn: 122.0000 - val_accuracy: 0.5755 - val_precision: 0.5423 - val_recall: 0.9730 - val_auc: 0.7198

Epoch 00005: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN_smalllr/abnormal/axial/weights.05.hdf5
Epoch 6/50
1646/1646 [==============================] - 757s 460ms/step - loss: 0.5496 - tp: 4828.6782 - fp: 3964.3542 - tn: 1022.9155 - fn: 137.5522 - accuracy: 0.5875 - precision: 0.5490 - recall: 0.9723 - auc: 0.7352 - val_loss: 0.6936 - val_tp: 5279.8999 - val_fp: 4186.6001 - val_tn: 1245.0000 - val_fn: 155.0000 - val_accuracy: 0.6005 - val_precision: 0.5577 - val_recall: 0.9715 - val_auc: 0.7475

Epoch 00006: saving model to /content/gdrive/My Drive/Colab Notebo

In [0]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(1,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)

# model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN/abnormal/axial/weights.20.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=50, initial_epoch = 20,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

## training on augmneted data

In [0]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(2,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}

batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_train , batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=1)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=class_count_test, batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=1)

model.load_weights('/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN/abnormal/axial/weights.20.hdf5')
model.fit(x=train_data_gen,validation_data=test_data_gen, epochs=30, initial_epoch = 20,verbose=True, callbacks=[cbs],shuffle=False, use_multiprocessing=True, workers=6)

# **testing on validation dataset**

In [5]:
label_type = 'abnormal'
exam_view = 'axial'
model, cbs = MRNet_inc_model(1,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception_BN/"+label_type+"/"+exam_view+"/weights.20.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])

neg = total - pos
class_count_valid = {0: neg, 1: pos}

batch_size = 1
print(model.metrics_names)
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , batch_size=1,model="inception",data_type='valid', shuffle=False,label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
d = model.evaluate_generator(valid_data_gen, total, verbose=1)
print(d)
model.predict_generator(valid_data_gen, 5, verbose=1)

Tensor("sequential_3/mr_net_inception_layer_3/dropout_3/cond/Identity:0", shape=(1, 2048), dtype=float32)
Tensor("sequential_3/mr_net_inception_layer_3_1/dropout_3/cond/Identity:0", shape=(1, 2048), dtype=float32)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_3 (MR (None, 1)                 21691809  
Total params: 21,691,809
Trainable params: 21,622,945
Non-trainable params: 68,864
_________________________________________________________________
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
Initializing Data Generator:
factor :  3.8
model:  inception
data type:  valid
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 23s 190ms/step
[0.6901630163192749, 47.95000076293945,

array([[0.50138813],
       [0.5014789 ],
       [0.501415  ],
       [0.50141776],
       [0.50149274]], dtype=float32)